# Data Processing and Cleaning

---
### Objective:

In this notebook, we will extract relevant miRNA expression data from the raw data file dump obtained from the National Cancer Institute (NCI) Genomics Data Commons data portal and create a working dataset to prepare it for visualization and modeling.

## Import Libraries

In [1]:
from IPython.display import clear_output
import os
import sys
import pandas as pd
import numpy as np
import time
import pickle

## Raw Data Processing

In [2]:
# Go to directory that contains the downloaded files
directory = '/Users/surajsakaram/GA_DSI/capstone/'
os.chdir(directory)

In [3]:
# Check to see that all your files have been downloaded
# There should be a total of 4,068 

len([i for i in os.listdir('./raw_data') if i != '.DS_Store'])

4068

In [4]:
os.listdir('./raw_data')[:5]

['dc8d96ef-81eb-41c8-a486-02d107c1a137',
 '93b8bab9-acd8-4a5a-b4ba-ec52b2ab186b',
 '6d869125-0f2b-406a-a1aa-4b352b2247d3',
 '41dbd6ab-871d-4ee4-be13-392b4995f6ce',
 '322badf4-b70d-4f90-ac44-d2f30e52f03c']

- These are directories named as the file [UUIDs](https://docs.gdc.cancer.gov/Data/Data_Model/GDC_Data_Model/) for the txt files that were downloaded.

#### Let's take a look at one of the data files we've downloaded.

In [5]:
# Set data file path and open it up as a dataframe 
path = './raw_data/46dc2547-0ffe-4bfa-ab8a-bfe04c9d1fc9/2c9af122-8b71-4e50-81b2-e95a5eeaeb03.mirbase21.mirnas.quantification.txt'

# Txt files are tab-delimited 
txt_file = pd.read_csv(path, delimiter='\t')
txt_file.head()

,miRNA_ID,read_count,reads_per_million_miRNA_mapped,cross-mapped
0,hsa-let-7a-1,56851,12062.449502,N
1,hsa-let-7a-2,56132,11909.894557,Y
2,hsa-let-7a-3,56890,12070.724388,N
3,hsa-let-7b,58020,12310.483898,N
4,hsa-let-7c,22880,4854.599648,Y


#### Goal: Extract out the `reads_per_million_miRNA_mapped` column from each file and store it as a dataframe.

In [6]:
# Create function that will return a list of file paths for the txt files we want to process

def fetch_file_paths(directory, ext='.mirnas.quantification.txt'):
    
    '''
    ** Description **
    Extract out files of your choosing from subdirectories by specifying file ending or extension.
    
    ** Parameters **
    directory: filepath to parent directory 
    ext: file extension or ending to search by
    '''
    
    file_paths = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if(file.endswith(ext)):
                file_paths.append(os.path.join(root, file))
    
    return file_paths

In [7]:
# Get list of file paths for txt files
file_path_list = fetch_file_paths('./raw_data')
file_path_list[0:2]

['./raw_data/dc8d96ef-81eb-41c8-a486-02d107c1a137/960e161b-c2da-496e-9aaf-f73a6e14ee46.mirbase21.mirnas.quantification.txt',
 './raw_data/93b8bab9-acd8-4a5a-b4ba-ec52b2ab186b/87d84d73-132c-44dc-9aa0-155665590242.mirbase21.mirnas.quantification.txt']

- This list of file paths will make it easy to access each text file and extract out the relevant columns. 

In [8]:
# Create a function that will return a dataframe with the relevant column extracted out from each file

def process_files(list_of_file_paths):
    # create a dataframe with the miRNA IDs using one of the files
    df = pd.read_csv(list_of_file_paths[0], delimiter='\t')
    df.drop([col for col in df.columns if col != 'miRNA_ID'], axis=1, inplace=True)
    
    n_files = len(list_of_file_paths)
    
    for i, file in enumerate(list_of_file_paths):
        ## Track progress ##
        clear_output(wait=True)
        print(f'Processing file {i}/{n_files}')
        
        # file manipulation
        sample_id = file.split('/')[1]
        tmp_df = pd.read_csv(file, delimiter='\t')
        tmp_df.rename(columns={'reads_per_million_miRNA_mapped': sample_id}, inplace=True)
        df = pd.merge(df, tmp_df[['miRNA_ID', sample_id]], how='outer', on='miRNA_ID')
    
    return df

**Note**: The below cell blocks are intentionally commented out so as not to be run each time the notebook is opened. The process of accessing each file and extracting out information takes a considerable amount of time. After the data has been processed, the resulting dataframe is pickled ([serialized](https://www.datacamp.com/community/tutorials/pickle-python-tutorial)). This allows us to easily load up the pickled dataframe object if the need should arise downstream. 

In [9]:
# %time
# df = process_files(file_path_list)

In [10]:
# # Serialize the dataframe object 
# pickle.dump(df, open('./data/processed_data.pk', 'wb'))

## Format the Dataframe

Let $(x_{i}, y_{i})^{n}_{i = 1}$ be a collection of n observations, where $x_{i}$ is a vector consisting of the expression levels of $m$ genes for the $i$-th observation, and $y_{i}$ is the class label or tumor type of the $i$-th observation, $i = 1, ..., n$.

Put simply, the rows should be the samples with the columns as the miRNA. The values of the matrix represent the expression level of a given miRNA in a given sample. Each sample would have its associated tumor diagnosis.

Later, in the modeling phase, the tumor type will be the target value to predict based on the gene expression data.

In [11]:
# Load in the pickled dataframe object 
miRNA_df = pickle.load(open('./data/processed_data.pk', 'rb'));

In [12]:
# View the dataframe
miRNA_df.head()

,miRNA_ID,dc8d96ef-81eb-41c8-a486-02d107c1a137,93b8bab9-acd8-4a5a-b4ba-ec52b2ab186b,6d869125-0f2b-406a-a1aa-4b352b2247d3,41dbd6ab-871d-4ee4-be13-392b4995f6ce,322badf4-b70d-4f90-ac44-d2f30e52f03c,d681c69f-3020-463b-90c0-bd9435e3f216,28665a83-f3fe-4ebc-a421-2ab1b40ec796,c5e98c47-acf7-4ebd-8674-3938bb4f5db7,c7fcb557-2a7c-41d2-92f1-7014b9f639d3,...,4e63c197-9a3c-4b2d-9efc-133ceab4bc40,dc0f20a0-b439-410e-b661-bdf65e3c17db,c1fc795e-9319-4483-9f13-ba466e839a31,d5fcc7c7-e3a7-4dd3-8c0e-512371adf47e,a2e4b2e1-e0c8-4a11-a1f3-cea88b35315c,eed5f427-c2bd-429d-97ac-cc0f4e01de76,12d172c8-5deb-447b-b651-ed10adff51d6,d7ecdf2e-4b2a-4beb-8d66-30c1fdb4a9f7,562b836f-8ced-4938-b21f-9d3d7409cc5a,57e7f0ba-8789-4c64-b000-df5a0db64f9e
0,hsa-let-7a-1,3530.574305,5860.628124,12710.168915,8612.209891,10317.399990,39795.949557,16039.138165,5666.609288,10979.201793,...,11734.336416,10890.841967,10132.541617,5499.395389,8337.330634,19997.810283,13385.740936,14877.067582,12861.693211,5711.236196
1,hsa-let-7a-2,3545.647914,5770.822052,12822.741990,8453.920962,10054.249488,39660.130365,15971.824671,5662.709733,10964.916514,...,11719.403012,10859.380777,10220.210971,5468.023000,8329.191163,19880.251089,13366.544012,14924.389366,12818.777248,5666.235889
2,hsa-let-7a-3,3516.338118,5873.585219,12923.054631,8830.260967,10189.749276,39721.766849,15966.266676,5610.344292,11015.108033,...,11913.122438,10940.349424,10236.628453,5563.524245,8378.078234,19987.187464,13332.263790,14968.894377,12840.393009,5764.475997
3,hsa-let-7b,13112.365384,32816.747358,8665.340311,33663.855692,24500.675020,94361.331089,7233.421582,16226.044899,51360.209476,...,19252.682724,9641.657043,15080.770729,18603.827155,14490.770819,18488.803478,45632.459805,13215.922729,19333.483539,22303.673483
4,hsa-let-7c,1015.793793,2379.637523,504.349668,1540.894268,3229.273903,3103.801144,2654.868896,991.600917,3571.319639,...,9526.889044,607.025300,2144.451515,3331.932341,6309.496959,6906.531801,1243.686440,3760.016210,7670.912823,867.681983


In [13]:
# Check dimensions 
miRNA_df.shape

(1881, 4069)

In [14]:
# Check to see if there are any missing values
miRNA_df.isna().sum().unique()

array([0])

In [15]:
# Set the miRNA_ID column as the index
miRNA_df.set_index('miRNA_ID', inplace=True)

In [16]:
# View the dataframe
miRNA_df.head(3)

,dc8d96ef-81eb-41c8-a486-02d107c1a137,93b8bab9-acd8-4a5a-b4ba-ec52b2ab186b,6d869125-0f2b-406a-a1aa-4b352b2247d3,41dbd6ab-871d-4ee4-be13-392b4995f6ce,322badf4-b70d-4f90-ac44-d2f30e52f03c,d681c69f-3020-463b-90c0-bd9435e3f216,28665a83-f3fe-4ebc-a421-2ab1b40ec796,c5e98c47-acf7-4ebd-8674-3938bb4f5db7,c7fcb557-2a7c-41d2-92f1-7014b9f639d3,66b21435-28b2-4721-b0c5-a51a03990240,...,4e63c197-9a3c-4b2d-9efc-133ceab4bc40,dc0f20a0-b439-410e-b661-bdf65e3c17db,c1fc795e-9319-4483-9f13-ba466e839a31,d5fcc7c7-e3a7-4dd3-8c0e-512371adf47e,a2e4b2e1-e0c8-4a11-a1f3-cea88b35315c,eed5f427-c2bd-429d-97ac-cc0f4e01de76,12d172c8-5deb-447b-b651-ed10adff51d6,d7ecdf2e-4b2a-4beb-8d66-30c1fdb4a9f7,562b836f-8ced-4938-b21f-9d3d7409cc5a,57e7f0ba-8789-4c64-b000-df5a0db64f9e
miRNA_ID,,,,,,,,,,,,,,,,,,,,,
hsa-let-7a-1,3530.574305,5860.628124,12710.168915,8612.209891,10317.399990,39795.949557,16039.138165,5666.609288,10979.201793,20656.884281,...,11734.336416,10890.841967,10132.541617,5499.395389,8337.330634,19997.810283,13385.740936,14877.067582,12861.693211,5711.236196
hsa-let-7a-2,3545.647914,5770.822052,12822.741990,8453.920962,10054.249488,39660.130365,15971.824671,5662.709733,10964.916514,20478.207813,...,11719.403012,10859.380777,10220.210971,5468.023000,8329.191163,19880.251089,13366.544012,14924.389366,12818.777248,5666.235889
hsa-let-7a-3,3516.338118,5873.585219,12923.054631,8830.260967,10189.749276,39721.766849,15966.266676,5610.344292,11015.108033,20949.000372,...,11913.122438,10940.349424,10236.628453,5563.524245,8378.078234,19987.187464,13332.263790,14968.894377,12840.393009,5764.475997


In [17]:
# Transpose the dataframe
miRNA_df_T = miRNA_df.T.rename_axis('file_ID', axis=1)
miRNA_df_T.columns.name = ''
miRNA_df_T.head(3)

,hsa-let-7a-1,hsa-let-7a-2,hsa-let-7a-3,hsa-let-7b,hsa-let-7c,hsa-let-7d,hsa-let-7e,hsa-let-7f-1,hsa-let-7f-2,hsa-let-7g,...,hsa-mir-941-5,hsa-mir-942,hsa-mir-943,hsa-mir-944,hsa-mir-95,hsa-mir-9500,hsa-mir-96,hsa-mir-98,hsa-mir-99a,hsa-mir-99b
dc8d96ef-81eb-41c8-a486-02d107c1a137,3530.574305,3545.647914,3516.338118,13112.365384,1015.793793,380.189927,608.806338,1039.241630,981.459460,285.561157,...,0.0,2.512268,0.000000,1.674845,2.512268,0.0,66.993820,42.708560,217.729914,27493.426231
93b8bab9-acd8-4a5a-b4ba-ec52b2ab186b,5860.628124,5770.822052,5873.585219,32816.747358,2379.637523,226.078969,395.861593,940.953178,928.442879,180.505738,...,0.0,1.787186,0.000000,4.021167,5.361557,0.0,35.296914,22.786615,702.363910,20079.922937
6d869125-0f2b-406a-a1aa-4b352b2247d3,12710.168915,12822.741990,12923.054631,8665.340311,504.349668,1599.429333,4876.308941,12166.251484,12349.600700,2128.299868,...,0.0,3.343755,1.114585,14.489604,3.343755,0.0,90.838669,136.536650,112.573075,43691.728108


## Log Transform the expression data

Log-transformation will allow us to model proportional changes rather than additive changes. This is biologically more relevant.

In [18]:
# Offset expression values by 1 to ensure that there are no 0's present in dataframe prior to log transformation
miRNA_df_T = miRNA_df_T.apply(lambda x: x + 1, axis=1)

In [19]:
# Check to see that there are no 0 values
miRNA_df_T[miRNA_df_T != 0].isna().sum().unique()

array([0])

In [20]:
# Log transform the data
miRNA_df_T_log = miRNA_df_T.apply(np.log10, axis=1)

In [21]:
# View the dataframe
miRNA_df_T_log.head(3)

,hsa-let-7a-1,hsa-let-7a-2,hsa-let-7a-3,hsa-let-7b,hsa-let-7c,hsa-let-7d,hsa-let-7e,hsa-let-7f-1,hsa-let-7f-2,hsa-let-7g,...,hsa-mir-941-5,hsa-mir-942,hsa-mir-943,hsa-mir-944,hsa-mir-95,hsa-mir-9500,hsa-mir-96,hsa-mir-98,hsa-mir-99a,hsa-mir-99b
dc8d96ef-81eb-41c8-a486-02d107c1a137,3.547968,3.549818,3.546214,4.117714,3.007233,2.581141,2.785192,3.017134,2.992315,2.457217,...,0.0,0.545588,0.000000,0.427299,0.545588,0.0,1.832469,1.640566,2.339908,4.439245
93b8bab9-acd8-4a5a-b4ba-ec52b2ab186b,3.768018,3.761313,3.768977,4.516109,3.376693,2.356177,2.598639,2.974029,2.968223,2.258890,...,0.0,0.445166,0.000000,0.700805,0.803563,0.0,1.559870,1.376333,2.847180,4.302784
6d869125-0f2b-406a-a1aa-4b352b2247d3,4.104185,4.108015,4.111399,3.937836,2.703592,3.204237,3.688180,4.085192,4.091688,3.328237,...,0.0,0.637865,0.325225,1.190040,0.637865,0.0,1.963026,2.138418,2.055275,4.640409


## Assign tumor classification

In [22]:
samples = pd.read_csv('./manifest/gdc_sample_sheet.2019-02-04.tsv', delimiter='\t')
samples.head()

,File ID,File Name,Data Category,Data Type,Project ID,Case ID,Sample ID,Sample Type
0,65f94233-e5fd-468f-aec6-94f2347d2f34,35a20c09-b72d-49f3-95b7-91ee3002fd8d.mirbase21...,Transcriptome Profiling,miRNA Expression Quantification,TCGA-BRCA,TCGA-W8-A86G,TCGA-W8-A86G-01A,Primary Tumor
1,dd81dd2f-0be1-4507-b931-9cc2b197b9f8,10db26c3-20bd-4204-bbdc-eb1ad49e91c3.mirbase21...,Transcriptome Profiling,miRNA Expression Quantification,TCGA-BRCA,TCGA-AR-A2LH,TCGA-AR-A2LH-01A,Primary Tumor
2,347a8e69-dab4-4920-a395-8ee80256d0ff,daf3b479-79cc-4f4e-b136-923ef6156e48.mirbase21...,Transcriptome Profiling,miRNA Expression Quantification,TCGA-BRCA,TCGA-E2-A1B5,TCGA-E2-A1B5-01A,Primary Tumor
3,406fd7d3-b73a-4014-b315-1eeaae01e988,ed031645-7a24-48a1-a0d5-70ea334abf5f.mirbase21...,Transcriptome Profiling,miRNA Expression Quantification,TCGA-BRCA,TCGA-A8-A07L,TCGA-A8-A07L-01A,Primary Tumor
4,2a3fd28d-255b-4079-8348-029bd0755803,589dd6e4-a03e-44bf-bc11-451602e4d22e.mirbase21...,Transcriptome Profiling,miRNA Expression Quantification,TCGA-BRCA,TCGA-B6-A0IK,TCGA-B6-A0IK-01A,Primary Tumor


In [23]:
merge_df = miRNA_df_T_log.reset_index().merge(samples[['File ID', 'Project ID']], \
                                          how='outer', left_on='index', right_on='File ID').set_index('index')
merge_df.drop('File ID', axis=1, inplace=True)
merge_df.index.names = ['']

In [24]:
merge_df.head(3)

,hsa-let-7a-1,hsa-let-7a-2,hsa-let-7a-3,hsa-let-7b,hsa-let-7c,hsa-let-7d,hsa-let-7e,hsa-let-7f-1,hsa-let-7f-2,hsa-let-7g,...,hsa-mir-942,hsa-mir-943,hsa-mir-944,hsa-mir-95,hsa-mir-9500,hsa-mir-96,hsa-mir-98,hsa-mir-99a,hsa-mir-99b,Project ID
,,,,,,,,,,,,,,,,,,,,,
dc8d96ef-81eb-41c8-a486-02d107c1a137,3.547968,3.549818,3.546214,4.117714,3.007233,2.581141,2.785192,3.017134,2.992315,2.457217,...,0.545588,0.000000,0.427299,0.545588,0.0,1.832469,1.640566,2.339908,4.439245,TCGA-BRCA
93b8bab9-acd8-4a5a-b4ba-ec52b2ab186b,3.768018,3.761313,3.768977,4.516109,3.376693,2.356177,2.598639,2.974029,2.968223,2.258890,...,0.445166,0.000000,0.700805,0.803563,0.0,1.559870,1.376333,2.847180,4.302784,TCGA-BRCA
6d869125-0f2b-406a-a1aa-4b352b2247d3,4.104185,4.108015,4.111399,3.937836,2.703592,3.204237,3.688180,4.085192,4.091688,3.328237,...,0.637865,0.325225,1.190040,0.637865,0.0,1.963026,2.138418,2.055275,4.640409,TCGA-LUAD


In [25]:
merge_df[['hsa-let-7a-1', 'hsa-let-7a-2', 'hsa-let-7a-3', 'hsa-let-7b', \
          'hsa-let-7c', 'hsa-let-7d', 'hsa-let-7e', 'Project ID']].head()

,hsa-let-7a-1,hsa-let-7a-2,hsa-let-7a-3,hsa-let-7b,hsa-let-7c,hsa-let-7d,hsa-let-7e,Project ID
,,,,,,,,
dc8d96ef-81eb-41c8-a486-02d107c1a137,3.547968,3.549818,3.546214,4.117714,3.007233,2.581141,2.785192,TCGA-BRCA
93b8bab9-acd8-4a5a-b4ba-ec52b2ab186b,3.768018,3.761313,3.768977,4.516109,3.376693,2.356177,2.598639,TCGA-BRCA
6d869125-0f2b-406a-a1aa-4b352b2247d3,4.104185,4.108015,4.111399,3.937836,2.703592,3.204237,3.688180,TCGA-LUAD
41dbd6ab-871d-4ee4-be13-392b4995f6ce,3.935165,3.927110,3.946023,4.527177,3.188055,2.909463,3.352680,TCGA-BRCA
322badf4-b70d-4f90-ac44-d2f30e52f03c,4.013612,4.002393,4.008206,4.389196,3.509239,2.801726,3.011989,TCGA-KIRC


## Save Dataframe as CSV

In [26]:
merge_df.to_csv('./data/expression.csv')